# Initialization

In [ ]:
import bgk

In [ ]:
# Parameters to adjust figures
B = .1
res = 512
case = ["exact", "max", "exact-rev", "max-rev"][0]

In [ ]:
path = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/{case}/B{B:05.2f}-n{res}/"

ve_coef = bgk.readParam(path, "v_e_coef", float)
input_path = bgk.readParam(path, "path_to_data", str)
exact_str = "Max" if bgk.readParam(path, "maxwellian", str).lower() == "true" else "Exact"

struct_radius = bgk.Input(input_path).get_radius_of_structure()

wholeSlice = bgk.DataSlice(slice(None, None), "")
centerSlice = bgk.DataSlice(slice(-struct_radius, struct_radius), "Central ")

loader = bgk.Loader(path, engine="pscadios2", species_names=['e', 'i'])
size = loader._get_xr_dataset("pfd", 0).length[1]

print(f"B={B}")
print(f"res={res}")
print(f"size={size}")
print(f"struct size={2*struct_radius:.3f}")
print(f"ve_coef={ve_coef}")
print(f"input_path={input_path}")

In [ ]:
# fiddle with this until as many steps as possible are used (usually, they can all be used)
nframes = 1000

videoMaker = bgk.VideoMaker(nframes, loader)

completion_percent = 100 * loader.fields_max / loader.nmax
video_coverage_percent = 100 * nframes * videoMaker.fields_stepsPerFrame / loader.fields_max
steps_used_percent = 100 * nframes / (loader.fields_max / loader.fields_every)
print(f"steps simulated:      {loader.fields_max} ({completion_percent:.1f}% complete)")
print(f"nframes in animation: {nframes}")
print(f"steps per frame:      {videoMaker.fields_stepsPerFrame}")
print(f"max step in video:    {nframes * videoMaker.fields_stepsPerFrame} ({video_coverage_percent:.1f}% coverage, {steps_used_percent:.1f}% step used)")
if video_coverage_percent != 100:
    print(f"suggested nframes:    {loader.get_all_suggested_nframes(nframes)[0]}")

# Load Data

In [ ]:
from IPython.display import HTML
import bgk.run_params as rp

In [ ]:
# select parameter
param = rp.e_phi
print(f"quantity: {param.title}")

In [ ]:
# load data
videoMaker.loadData(param)
videoMaker.setSlice(wholeSlice)

### Check Frame/Video

In [ ]:
# view t=0
%matplotlib widget
fig, ax, im = videoMaker.viewFrame(3)
fig

In [ ]:
# make movie
%matplotlib widget
anim = videoMaker.viewMovie(fig, ax, im)
HTML(anim.to_html5_video())

# Spiral

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

In [ ]:
maxR = size / 2
maxT = videoMaker.times[-1]
rStart = struct_radius / 2
rStop = struct_radius * 2

thetaGrid = videoMaker.xGrid

def getRslice(data: xr.DataArray) -> xr.DataArray:
    return data.where((rStart <= videoMaker.rGrid) & (videoMaker.rGrid < rStop))

def flattenRslice(rslice: xr.DataArray) -> np.ndarray:
    rslice = rslice.data.flatten()
    return rslice[~np.isnan(rslice)]

## Choose Time

In [ ]:
t = 10

idx = int(t / maxT * nframes)

ne = getRslice(videoMaker.slicedDatas[idx])
theta = getRslice(np.arctan2(ne.z, ne.y))
print(idx)

In [ ]:
%matplotlib inline
plt.close("all")
im = plt.imshow(
            # theta,
            ne,
            origin="lower",
            extent=(
                videoMaker._currentSlice.slice.start,
                videoMaker._currentSlice.slice.stop,
                videoMaker._currentSlice.slice.start,
                videoMaker._currentSlice.slice.stop,
            ),)
plt.colorbar(im)

In [ ]:
theta_axis = flattenRslice(theta)
ne_axis = flattenRslice(ne)

### Scatter at time

In [ ]:
plt.close("all")
plt.scatter(theta_axis, ne_axis, s=10)

### Binning at time

In [ ]:
theta_bins = np.linspace(theta_axis.min(), theta_axis.max(), 100)
bin_idxs = np.digitize(theta_axis, theta_bins)
ne_means = [ne_axis[bin_idxs == i].mean() for i in range(1, len(theta_bins)+1)]

In [ ]:
plt.close("all")
plt.plot(theta_bins, ne_means)

### Fit at time

In [ ]:
import scipy.optimize as opt

In [ ]:
theta_bins
ne_means

def f_ephi(theta, amp, freq1, freq2, phase1, phase2, offset) -> float:
    # return amp * np.sin(theta * freq1 + phase1) * np.sin(theta * freq2 + phase2)**2 + offset
    return amp * np.sin(theta * freq1 + phase1) * np.abs(np.sin(theta * freq2 + phase2))**2 + offset

f = f_ephi

[popt, pcov] = opt.curve_fit(f, theta_bins, ne_means, p0=[3e-5, 8, 1, 0, 0, 0])
perr = np.sqrt(np.diag(pcov))

print(popt)
print(perr)

In [ ]:
plt.close("all")
plt.plot(theta_bins, ne_means)
plt.plot(theta_bins, [f(theta, *popt) for theta in theta_bins])

### FFT at time

In [ ]:
import scipy.signal as sig

In [ ]:
idx_freq, power = sig.periodogram(ne_means, nfft=len(ne_means) * 4)
freq = idx_freq * len(theta_bins) / 6.28

In [ ]:
plt.close("all")
plt.xlabel("Frequency")
plt.ylabel("Amplitude")
plt.plot(freq, power)

In [ ]:
search_cutoff = 0 #len(power) // 2
peak_freq = freq[power[search_cutoff:].argmax() + search_cutoff]
print(f"peak frequency: {peak_freq}")

## Arbitrary Shift Finder

In [ ]:
thetas = np.linspace(-np.pi, np.pi, 100)
rslice = getRslice(videoMaker.slicedDatas[0])
bin_idxs = np.digitize(flattenRslice(getRslice(np.arctan2(rslice.z, rslice.y))), thetas)

In [ ]:
def interpolate_in_nes(thetas: np.ndarray, nes: np.ndarray, theta: float) -> float:
    idx = (thetas > theta).argmax()
    dtheta1 = thetas[idx] - theta
    dtheta2 = theta - thetas[idx-1]
    return (nes[idx] * dtheta2 + nes[idx-1] * dtheta1) / (dtheta1 + dtheta2)

def find_dtheta(thetas: np.ndarray, nes1: np.ndarray, nes2: np.ndarray, n_iterations=3, shift_search=None, max_shift = np.pi/8):
    best_shift = np.nan
    best_sum_square_errors = np.inf
    shift_search = shift_search if shift_search is not None else np.linspace(-max_shift, max_shift, len(thetas))
    for shift in shift_search:
        this_sum_square_errors = 0
        for theta, ne2 in zip(thetas, nes2):
            this_sum_square_errors += (interpolate_in_nes(thetas, nes1, theta + shift) - ne2) ** 2
        if this_sum_square_errors < best_sum_square_errors:
            best_shift, best_sum_square_errors = shift, this_sum_square_errors

    if n_iterations <= 1:
        # Error is sum_theta |nes1(theta+shift) - nes2(theta)|^2
        # so shift > 0 means nes1 and nes2 match when nes1 is rotated clockwise, ie, delta_theta < 0
        return -best_shift

    dtheta = shift_search[1] - shift_search[0]
    return find_dtheta(thetas, nes1, nes2, n_iterations-1, np.linspace(best_shift - 4*dtheta, best_shift + 4*dtheta, 10))

def get_nes(bin_idxs: np.ndarray, frame: int) -> np.ndarray:
    ne_axis = flattenRslice(getRslice(videoMaker.slicedDatas[frame]))
    return [ne_axis[bin_idxs == i].mean() for i in range(1, max(bin_idxs) + 1)]

def t2i(t: float) -> int:
    return int(t * nframes / max(videoMaker.times))

In [ ]:
# Find dthetas
nes2 = get_nes(bin_idxs, 0)
dthetas = []
for frame in range(1, nframes):
    nes1, nes2 = nes2, get_nes(bin_idxs, frame)
    dthetas.append(find_dtheta(thetas, nes1, nes2, n_iterations=5))

dthetas = np.array(dthetas)

phase_velocities = dthetas / videoMaker.times[1]

phase_angles = [0]
for shift in dthetas:
    phase_angles.append(phase_angles[-1] + shift)

In [ ]:
# plot phase angle vs time
plt.close("all")
plt.plot(videoMaker.times, phase_angles)
plt.xlabel("Time")
plt.ylabel("Phase Angle")
plt.title(f"Motion of Spiral in {param.title} ($B_0={B}$, {exact_str})")
plt.show()

### Linear Fit

In [ ]:
import scipy.stats as stats

In [ ]:
# Choose time frame
tstart = 25
tstop = 70
istart, istop = t2i(tstart), t2i(tstop)+1

fit = stats.linregress(videoMaker.times[istart:istop], phase_angles[istart:istop])

In [ ]:
# plot phase angle vs time
plt.close("all")
fig, ax = plt.subplots(1,1)
ax.plot(videoMaker.times, phase_angles, label="From run")
ax.plot(videoMaker.times[istart:istop], [fit.intercept + fit.slope * t for t in videoMaker.times[istart:istop]], label=f"Linear fit ($d\phi/dt={fit.slope:.3f}$, $r^2={fit.rvalue**2:.3f}$)")
ax.axvline(videoMaker.times[istart], color="grey", linestyle="--")
ax.axvline(videoMaker.times[istop-1], color="grey", linestyle="--")
ax.set_xlabel("Time")
ax.set_ylabel("Phase Angle, $\phi$ (rad)")
ax.set_title(f"Motion of Spiral in {param.title} ($B_0={B}$, {exact_str})")
ax.legend()
plt.show()

### Compare to Phidot of Electrons

In [ ]:
input = bgk.input_reader.Input(f"../psc/inputs/bgk/case1-B={B}-input.txt")

phidot = input.v_phi[1:] / input.rho[1:]

print(f"peak electron phidot = {phidot.min()}")

### Save Figure

In [ ]:
import os

In [ ]:
def get_fig_path(outdir, fig_name: str) -> str:
    return os.path.join(outdir, fig_name)

def get_fig_name() -> str:
    param_str = param.title.replace("_", "").replace("$", "").replace("\\", "").lower()
    return f"spiral-{param_str}-{case}-B{B:05.2f}-n{res}.png"

In [ ]:
# Save Figure
outdir = f"/mnt/lustre/IAM851/jm1667/psc-scrap/figs-{case}/"
os.makedirs(outdir, exist_ok=True)

fig_path = get_fig_path(outdir, get_fig_name())
fig.savefig(fig_path, bbox_inches="tight", pad_inches=0.01, dpi=300)
print(f"saved to {fig_path}")